In [1]:
from auto_functions import *
import yaml
from py_sc_fermi.inputs import inputs_from_files
from pandas import DataFrame as df
import pandas as pd
from itertools import product
from tqdm import tqdm
import numpy as np
from scipy.constants import physical_constants

with open('automator_config.yaml', 'r') as stream:
    try:
        a = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

defects = import_calculations_from_file('defects.yaml')
elements = import_calculations_from_file('elements.yaml')
interest = import_calculations_from_file('interest.yaml')
        

def make_all_defects(config_dict,chem_pots):
    to_write = []
    defs = []
    for k,v in config_dict['defects'].items():
        label = [i for i in config_dict['defects'][k][1:]]
        to_calcs = [defects[i] for i in label]
        out = make_defect(to_calcs, elements, interest['LLZO'], delta_mu=chem_pots, corr=config_dict['iccs'], sites=1)
        out._nsites = config_dict['defects'][out.name][0]
        defs.append(out)
    return defs

ex_grid = (pd.read_csv('ex_grid.dat', skiprows=10, sep='\s+'))
grid = (pd.read_csv('grid.dat', skiprows=10, sep='\s+'))
all_defects = make_all_defects(a,ex_grid.iloc[5497].to_dict())

In [2]:
### The idea in this cell is to replicate the unrelaxed calculation, i.e. high T, nothing fixed

unitcell_filename = 'unitcell.dat'
totdos_filename = 'totdos.dat'
input_fermi_filename = 'input-fermi.dat'

inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)


defect_system_unrelaxed = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=1500)

#defect_system.report()
constraint = (defect_system_unrelaxed.to_dict()['v_Li']) - (defect_system_unrelaxed.to_dict()['Li_i'])

defect_system_unrelaxed.report()

## These carrier concentrations are the same as Aron and Sunghyun's, so, I assume this is all still working 

Volume of cell: 1056.622886347127 A^3
Found spin polarised system...
Number of electrons in system: 544
Energy gap of system: 5.904 eV
Temperature: 1500.0 K
Number of defect species: 13
Reading spin-polarised DOS
SC Fermi level :      3.378588089859929  (eV)

Concentrations:
n (electrons)  : 4847943608420.384 cm^-3
p (holes)      : 44482920693.26799 cm^-3
v_O            : 86607208011835.23 cm^-3
v_Li           : 3.168454505025838e+19 cm^-3
Li_i           : 3.1689368227470954e+19 cm^-3
O_i            : 104811.65773031449 cm^-3
v_La           : 223517456786.10812 cm^-3
v_Zr           : 157591.95201718222 cm^-3
Zr_i           : 1401348029.2747295 cm^-3
Zr_Li_tet      : 123190077990.1366 cm^-3
Zr_Li          : 2176065384489526.8 cm^-3
Zr_La          : 3062494056181169.0 cm^-3
La_Zr          : 900328195929401.9 cm^-3
Li_La          : 4344991936475145.0 cm^-3
Li_Zr          : 1652075880227931.8 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1763: RuntimeWarning: invalid value encountered in double_scalars
  q = 2.0 * (q - r)


In [3]:
# This cell is an excercise in what happens if we just let everything fall to 300 K with nothing fixed, essentially everything other then Li defects are totally wiped out 
defect_system_full_relax = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300)
defect_system_full_relax.report()

SC Fermi level :      3.435400610912699  (eV)

Concentrations:
n (electrons)  : 1.410777787112445e-21 cm^-3
p (holes)      : 2.8920215984735876e-37 cm^-3
v_O            : 1.0282294020879069e-19 cm^-3
v_Li           : 4424250209729.337 cm^-3
Li_i           : 4424258787811.561 cm^-3
O_i            : 1.5766234074935817e-59 cm^-3
v_La           : 3.1720433505619897e-26 cm^-3
v_Zr           : 7.961214104162664e-55 cm^-3
Zr_i           : 1.0168276604181081e-42 cm^-3
Zr_Li_tet      : 2.7266172353264954e-32 cm^-3
Zr_Li          : 5.2053723899453675e-12 cm^-3
Zr_La          : 2.330844971766325e-09 cm^-3
La_Zr          : 4.1492671933788866e-10 cm^-3
Li_La          : 9.779563656522404e-06 cm^-3
Li_Zr          : 1.1195706995312348e-05 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2  9.969931e-20          96.96 
           :  1  3.123630e-21          3.04 
      

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1760: RuntimeWarning: invalid value encountered in double_scalars
  r = (xf - nfc) * (fx - ffulc)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy

In [4]:
### This cell is where we start cheating the system by fixing things. This begins by fixing everything to the concentrations in the unrelaxed calculation

from py_sc_fermi.defect_charge_state import DefectChargeState, FrozenDefectChargeState
from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.inputs import inputs_from_files

all_frozen_charge_states = {}
for i in all_defects:
    frozen_charge_states = {}
    for j in i.charge_states:
        c = i.charge_states[j].get_concentration(defect_system_unrelaxed.get_sc_fermi(verbose=False), defect_system_unrelaxed.temperature)
        frozen_charge_state = FrozenDefectChargeState(charge= j , concentration=c)
        frozen_charge_states.update({j : frozen_charge_state})
    all_frozen_charge_states.update({i.name:frozen_charge_states})
for x in all_defects:
     x._charge_states = None
     x._charge_states = all_frozen_charge_states[x.name]
#print(all_defects)

test_defect_system = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300)
print(test_defect_system.to_dict())      ###
print(defect_system_unrelaxed.to_dict()) ### This is just a sanity check comparison, to see if all the concentrations remain the same after fixing as they do before fixing

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1763: RuntimeWarning: invalid value encountered in double_scalars
  q = 2.0 * (q - r)


{'Fermi Energy': 2.8476878124222647, 'p0': 2.1593888043346525e-27, 'n0': 1.8894234437012732e-31, 'v_O': 86607208011835.23, 'v_Li': 3.168454505025838e+19, 'Li_i': 3.1689368227470954e+19, 'O_i': 104811.65773031449, 'v_La': 223517456786.10812, 'v_Zr': 157591.95201718222, 'Zr_i': 1401348029.2747295, 'Zr_Li_tet': 123190077990.1366, 'Zr_Li': 2176065384489526.8, 'Zr_La': 3062494056181169.0, 'La_Zr': 900328195929401.9, 'Li_La': 4344991936475145.0, 'Li_Zr': 1652075880227931.8}
{'Fermi Energy': 3.4129985636117595, 'p0': 34086257625.09678, 'n0': 6326616823932.944, 'v_O': 86607208011835.23, 'v_Li': 3.168454505025838e+19, 'Li_i': 3.1689368227470954e+19, 'O_i': 104811.65773031449, 'v_La': 223517456786.10812, 'v_Zr': 157591.95201718222, 'Zr_i': 1401348029.2747295, 'Zr_Li_tet': 123190077990.1366, 'Zr_Li': 2176065384489526.8, 'Zr_La': 3062494056181169.0, 'La_Zr': 900328195929401.9, 'Li_La': 4344991936475145.0, 'Li_Zr': 1652075880227931.8}


/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))


In [5]:
# So now to try and replicate Sunghyun's analysis:

# first, reset the lithium defects
all_defects[1] = make_defect([defects['v_Li'],defects['v_Li_-']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=3)
all_defects[2] = make_defect([defects['Li_i'],defects['Li_i_+']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=1)

# then regenerate the defect system, 300K
defect_system_sunghyun = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300)

# then fix the total defect concentrations of Li defects to be those of the high T calc
defect_system_sunghyun.defect_species_by_name('v_Li').fix_concentration(defect_system_unrelaxed.to_dict()['v_Li'] / 1e24 * defect_system_sunghyun.volume)
defect_system_sunghyun.defect_species_by_name('Li_i').fix_concentration(defect_system_unrelaxed.to_dict()['Li_i'] / 1e24 * defect_system_sunghyun.volume)
defect_system_sunghyun.report()

### This calculation seems to explode. 

SC Fermi level :      -27.577780507626215  (eV)

Concentrations:
n (electrons)  : 0.0 cm^-3
p (holes)      : 4.76915363262079e+23 cm^-3
v_O            : 86607208011835.23 cm^-3
v_Li           : 3.1684544751421088e+19 cm^-3
Li_i           : 3.168936271464413e+19 cm^-3
O_i            : 104811.65773031449 cm^-3
v_La           : 223517456786.10812 cm^-3
v_Zr           : 157591.95201718222 cm^-3
Zr_i           : 1401348029.2747295 cm^-3
Zr_Li_tet      : 123190077990.1366 cm^-3
Zr_Li          : 2176065384489526.8 cm^-3
Zr_La          : 3062494056181169.0 cm^-3
La_Zr          : 900328195929401.9 cm^-3
Li_La          : 4344991936475145.0 cm^-3
Li_Zr          : 1652075880227931.8 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2  5.547053e+13          64.05  [fixed]
           :  1  3.113138e+13          35.95  [fixed]
           :  0  5.297193e+09          0.0

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_species.py:221: RuntimeWarning: invalid value encountered in double_scalars
  cs_concentrations[q] *= scaling


In [6]:
## reset v_li and li_i for ben-style analysis

all_defects[1] = make_defect([defects['v_Li'],defects['v_Li_-']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=3)
all_defects[2] = make_defect([defects['Li_i'],defects['Li_i_+']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=1)

defect_system_ben = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300)

defect_system_ben.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, (constraint / 1e24 * defect_system_ben.volume))

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-package

3.254589333978444